In [ ]:
# !pip3 install open3d --break-system-packages
# !pip3 install plyfile --break-system-packages
# !pip3 install pyntcloud --break-system-packages

# Import Library

In [ ]:
import os
import sys
import ast
import copy

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from plyfile import PlyData, PlyElement

from scipy.spatial import ConvexHull
from pyntcloud import PyntCloud

from Utilities.utilities import read_points3D, write_ply, calculate_volume, calculate_volume_file

# Function Plot

In [ ]:
def plot_mesh(mesh):
    vertices = np.asarray(mesh.vertices)
    
    # Plot the PLY file
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=0.005)
    
    # Set axis labels
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    # Set title
    ax.set_title('3D Mesh Plot')

    plt.show()
    return None

# User Defined Function

In [ ]:
#################################################
# Remove isolated mesh
#################################################
def remove_isolated_vertices(mesh, eps=1.0, min_points=10):
    print("\nRemoving isolated vertices using DBSCAN clustering")

    # Convert mesh to point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices

    # Apply DBSCAN clustering
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))

    # Keep only the largest cluster
    largest_cluster = np.argmax(np.bincount(labels[labels >= 0]))
    mask = labels == largest_cluster

    # Map old indices to new indices
    index_map = -np.ones(len(mask), dtype=int)
    index_map[mask] = np.arange(np.sum(mask))

    # Filter vertices
    filtered_vertices = np.asarray(mesh.vertices)[mask]

    # Filter triangles
    triangles = np.asarray(mesh.triangles)
    filtered_triangles = []
    for tri in triangles:
        if np.all(mask[tri]):
            filtered_triangles.append(index_map[tri])

    # Create cleaned mesh
    cleaned_mesh = o3d.geometry.TriangleMesh()
    cleaned_mesh.vertices = o3d.utility.Vector3dVector(filtered_vertices)
    cleaned_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)

    print(f"Remaining vertices: {len(filtered_vertices)}")
    return cleaned_mesh


In [ ]:
#################################################
# Reorientate Mesh (kinda works)
#################################################
def find_floor_plane_and_reorient(mesh, iteration=10):
    print(f'\nLocating floor plane and reorientate to x-y plane')

    for i in range(iteration):
        # Convert mesh vertices to numpy array
        points = np.asarray(mesh.vertices)

        # Create a PointCloud object from the mesh vertices
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)

        # Use RANSAC to find the plane in the point cloud
        plane_model, inliers = point_cloud.segment_plane(
            distance_threshold=0.01,
            ransac_n=3,
            num_iterations=1000)
        
        # Extract the plane coefficients
        a, b, c, d = plane_model
        
        # Calculate the normal vector of the plane
        normal_vector = np.array([a, b, c])
        
        # Calculate the rotation matrix to align the normal vector with the z-axis
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(normal_vector, z_axis)
        rotation_angle = np.arccos(np.dot(normal_vector, z_axis))
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(
            rotation_axis * rotation_angle)
        
        # Rotate the mesh
        mesh.rotate(rotation_matrix, center=(0, 0, 0))
        
        # Translate the mesh to ensure the center of the floor is at (0, 0, 0)
        points = np.asarray(mesh.vertices)

        floor_center = points[inliers].mean(axis=0)
        translation_vector = -floor_center
        mesh.translate(translation_vector)
        
        # Ensure all points are on the positive side of z-axis by rotating about the x-y plane
        points = np.asarray(mesh.vertices)

        min_z = points[:, 2].min()
        if min_z < 0:
            rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz([np.pi, 0, 0])
            mesh.rotate(rotation_matrix, center=(0, 0, 0))
        
        # To handle mesh and point cloud
        mean_z = np.mean(np.asarray(mesh.vertices)[:, 2]) 
        if mean_z < 0:
            rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz([np.pi, 0, 0])
            mesh.rotate(rotation_matrix, center=(0, 0, 0))
            
        # print(f'Rotation {i+1}')
        # print(np.round(rotation_matrix), 5)
        # print(rotation_matrix)
        
    print(f'No of points: {len(np.asarray(mesh.vertices))}')
        
    return mesh

In [ ]:
#################################################
# Remove floor
#################################################
# def crop_mesh_z_axis(mesh, percentile = 10):
#     print(f'\nCropping mesh {percentile}% off the base')
    
#     # Convert mesh vertices to numpy array
#     points = np.asarray(mesh.vertices)
#     triangles = np.asarray(mesh.triangles)

#     # Step 1: Shift all z-values to be positive
#     min_z = np.min(points[:, 2])
#     points[:, 2] -= min_z
    
#     # Compute a dynamic z-threshold using a low percentile (e.g., 5th percentile)
#     z_values = points[:, 2]
#     z_threshold = np.percentile(z_values, percentile)

#     # Create a mask to filter out points below the z_threshold
#     mask = points[:, 2] > z_threshold
    
#     # Apply the mask to get the filtered points and update the triangles accordingly
#     filtered_points = points[mask]
#     filtered_triangles = []
#     for triangle in triangles:
#         if np.all(mask[triangle]):
#             filtered_triangles.append(triangle)
    
#     # Create a new mesh with the filtered points and triangles
#     cropped_mesh = o3d.geometry.TriangleMesh()
#     cropped_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
#     cropped_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)
    
#     print(f'No of points: {len(np.asarray(cropped_mesh.vertices))}')
    
#     return cropped_mesh


#################################################
# Remove floor
#################################################
def crop_mesh_z_axis(mesh, percentile):
    print('\nCropping mesh below the bottom 10th percentile of Z values')

    # Convert mesh vertices to numpy array
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Calculate Z range and threshold
    z_values = points[:, 2]
    z_min, z_max = np.min(z_values), np.max(z_values)
    z_range = z_max - z_min
    z_threshold = z_min + percentile / 100 * z_range # bottom 10% threshold

    # Create a mask to keep points with Z >= threshold
    mask = z_values >= z_threshold

    # Map old indices to new indices
    index_map = -np.ones(len(points), dtype=int)
    index_map[mask] = np.arange(np.sum(mask))

    # Filter points
    filtered_points = points[mask]

    # Filter triangles: keep only those where all vertices are retained
    filtered_triangles = []
    for triangle in triangles:
        if np.all(mask[triangle]):
            new_triangle = index_map[triangle]
            filtered_triangles.append(new_triangle)

    # Create a new mesh
    cropped_mesh = o3d.geometry.TriangleMesh()
    cropped_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    cropped_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)

    print(f'No of points: {len(filtered_points)}')
    return cropped_mesh


In [ ]:
#################################################
# Remove smallest objects
#################################################
def remove_small_objects(mesh):
    print(f'\nPerform mesh clustering to remove small objects')
    
    for _ in range(3):
        # Convert mesh vertices to numpy array
        points = np.asarray(mesh.vertices)
        triangles = np.asarray(mesh.triangles)
        
        # Create a PointCloud object from the mesh vertices
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)
        
        # Cluster the points using DBSCAN algorithm
        labels = np.array(point_cloud.cluster_dbscan(eps=0.02, min_points=10))
        
        # Check the number of clusters
        num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        # If only one cluster, return the one
        if num_clusters <= 1:
            print(f'No of points: {len(np.asarray(mesh.vertices))}')
            return mesh
        
        # Find the largest cluster
        largest_cluster_label = np.argmax(np.bincount(labels[labels >= 0]))
        
        # Create a mask to filter out points that are not in the largest cluster
        mask = labels == largest_cluster_label
        
        # Apply the mask to get the filtered points
        filtered_points = points[mask]
        
        # Remap the indices of the triangles to the filtered points
        index_map = np.zeros(len(points), dtype=int) - 1
        index_map[mask] = np.arange(np.sum(mask))
        filtered_triangles = []
        for triangle in triangles:
            if np.all(triangle < len(mask)) and np.all(mask[triangle]):
                filtered_triangles.append(index_map[triangle])
        
        # Create a new mesh with the filtered points and triangles
        largest_object_mesh = o3d.geometry.TriangleMesh()
        largest_object_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
        largest_object_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)
        
        # If the amount of mesh removed is over 10% of the original mesh, stop the removal process    
        if len(np.asarray(largest_object_mesh.vertices)) * 10 < len(np.asarray(mesh.vertices)):
            print(f'No of points: {len(np.asarray(mesh.vertices))}')
            return mesh
        
        mesh = copy.deepcopy(largest_object_mesh)
        
    print(f'No of points: {len(np.asarray(largest_object_mesh.vertices))}')
    
    return largest_object_mesh

In [ ]:
#################################################
# Rescaling based on feet
#################################################
def scale_longest_xy_axis(mesh, target_length, percentile=10):
    print(f'\nScaling longest x-y axis based on foot length: {target_length}cm')
    
    # Convert mesh vertices to numpy array
    points = np.asarray(mesh.vertices)
    
    # Determine the threshold for the bottom 5th percentile of z-values
    z_threshold = np.percentile(points[:, 2], percentile)

    # Filter points to only include those with z-values in the bottom 5th percentile
    bottom_points = points[points[:, 2] <= z_threshold]

    # Compute the longest distance between any two points in the x-y plane
    longest_distance = 0
    point1 = None
    point2 = None
    for i in range(len(bottom_points)):
        for j in range(i + 1, len(bottom_points)):
            # Only consider pairs aligned along x or y axis
            if (
                np.isclose(bottom_points[i, 0], bottom_points[j, 0]) or 
                np.isclose(bottom_points[i, 1], bottom_points[j, 1])
                ):
                distance = np.linalg.norm(bottom_points[i, :2] - bottom_points[j, :2])
                if distance > longest_distance:
                    longest_distance = distance
                    point1 = bottom_points[i]
                    point2 = bottom_points[j]

    # Compute scaling factor
    scale = target_length / longest_distance / np.sqrt(2)
    print(f'Scaling Factor: {round(scale, 3)} - Longest Distance: {round(longest_distance, 3)} - Target Length: {target_length}')
    
    # Scale all coordinates
    points *= scale
    mesh.vertices = o3d.utility.Vector3dVector(points)
    
    print(f'No of points: {len(np.asarray(mesh.vertices))}')
    
    return mesh, point1, point2

In [ ]:
#################################################
# Reorientate feet (Take previous 2 points to orientate)
#################################################
def reorient_feet(mesh, point1, point2, target_direction=np.array([0, 1])):
    print(f'\nReorientate feet based on feet direction')
    
    points = np.asarray(mesh.vertices)

    # Compute the direction vector from point1 to point2 in the x-y plane
    direction_vector = point2[:2] - point1[:2]
    direction_vector /= np.linalg.norm(direction_vector)  # Normalize

    # Compute angle to target direction
    angle = np.arccos(np.dot(direction_vector, target_direction))

    # Determine rotation direction using cross product
    if np.cross(direction_vector, target_direction) < 0:
        angle = -angle

    # Rotation matrix around z-axis
    R = np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0,              0,             1]
    ])

    # Apply rotation
    rotated_points = points @ R.T
    mesh.vertices = o3d.utility.Vector3dVector(rotated_points)
    
    print(f'No of points: {len(np.asarray(mesh.vertices))}')
    
    return mesh


In [ ]:
#################################################
# Crop Ankle
#################################################
def crop_mesh_z_axis_ankle(mesh, z_threshold = 10):
    print(f'\nCropping mesh from the top, leaving {z_threshold}cm')
    
    # Convert mesh vertices to numpy array
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    
    # Create a mask to filter out points below the z_threshold
    mask = points[:, 2] < z_threshold
    
    # Apply the mask to get the filtered points and update the triangles accordingly
    filtered_points = points[mask]
    filtered_triangles = []
    for triangle in triangles:
        if np.all(mask[triangle]):
            filtered_triangles.append(triangle)
    
    # Create a new mesh with the filtered points and triangles
    cropped_mesh = o3d.geometry.TriangleMesh()
    cropped_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    cropped_mesh.triangles = o3d.utility.Vector3iVector(filtered_triangles)
    
    print(f'No of points: {len(np.asarray(cropped_mesh.vertices))}')
    
    return cropped_mesh

In [ ]:
#################################################
# Solid mesh
#################################################
def solidify_mesh_with_fans(mesh, z_min=0.0, z_max=10.0):
    print(f'\nSolidify mesh')
    points = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Keep only points within the z bounds
    mask = (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
    valid_indices = np.where(mask)[0]
    index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(valid_indices)}
    filtered_points = points[mask]

    # Remap triangle indices
    filtered_triangles = []
    for tri in triangles:
        if all(idx in index_map for idx in tri):
            new_tri = [index_map[idx] for idx in tri]
            filtered_triangles.append(new_tri)

    cap_triangles = []

    # Cap both ends using triangle fans
    for z_target in [z_min, z_max]:
        cap_indices = np.where(np.isclose(filtered_points[:, 2], z_target))[0]
        if len(cap_indices) >= 3:
            cap_ring = filtered_points[cap_indices]
            center = np.mean(cap_ring, axis=0)
            center_idx = len(filtered_points)
            filtered_points = np.vstack([filtered_points, center])

            # Sort cap points in counter-clockwise order
            xy = cap_ring[:, :2] - center[:2]
            angles = np.arctan2(xy[:, 1], xy[:, 0])
            sorted_indices = cap_indices[np.argsort(angles)]

            for i in range(len(sorted_indices)):
                i1 = sorted_indices[i]
                i2 = sorted_indices[(i + 1) % len(sorted_indices)]
                if z_target == z_min:
                    cap_triangles.append([i1, i2, center_idx])
                else:
                    cap_triangles.append([i2, i1, center_idx])  # flip normal

    all_triangles = np.vstack([filtered_triangles, cap_triangles])

    solid_mesh = o3d.geometry.TriangleMesh()
    solid_mesh.vertices = o3d.utility.Vector3dVector(filtered_points)
    solid_mesh.triangles = o3d.utility.Vector3iVector(all_triangles)
    solid_mesh.compute_vertex_normals()
    
    print(f'No of points: {len(np.asarray(solid_mesh.vertices))}')
    
    return solid_mesh



In [ ]:
def plot_all_meshes(
    subject, data_image, feet_length, file_name, volume, 
    point1, point2,
    meshes, list_title):
    
    if file_name == 'File_Path_SFM':
        s = 1
    else:
        s = 0.008
    
    num_meshes = len(meshes)
    # fig = plt.figure(figsize=(15, 5 * num_meshes))
    fig = plt.figure(figsize=(16, 10))
    
    for i, mesh in enumerate(meshes):
        # ax = fig.add_subplot(1, num_meshes, i + 1, projection='3d')
        ax = fig.add_subplot(2, 3, i + 1, projection='3d')
        vertices = np.asarray(mesh.vertices)
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=s)
        

        if list_title[i] == 'Rescaled':
            ax.scatter(
                [point1[0], point2[0]], 
                [point1[1], point2[1]], 
                [point1[2], point2[2]], 
                s=s*10, color='red', label='Longest Distance Points')
            
            ax.plot(
                [point1[0], point2[0]], 
                [point1[1], point2[1]], 
                [point1[2], point2[2]], 
                color='green', label='Longest Distance Line')
        
        # Set axis labels
        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Z axis')
        
        # Set title
        # ax.set_title(f'3D Mesh Plot {i + 1}')
        ax.set_title(f'({i+1}) {list_title[i]}\nPoints: {len(vertices)}')
    
    
    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {feet_length}cm - Points: {len(vertices)}\nVolume: {round(volume, 1)}cm3')
    plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {feet_length}cm - Points: {len(vertices)}')
    plt.tight_layout()
    # fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.3)

    folder_save_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Details/Feet_Preprocessing/{file_name}'
    if not os.path.exists(folder_save_path):
        os.makedirs(folder_save_path)

    save_path = f'{folder_save_path}/subject_{subject}_{data_image}.png'
    fig.savefig(save_path, dpi=300)  # Save at high resolution
    plt.close()
    # plt.show()

In [ ]:
def plot_mesh_angles(subject, data_image, feet_length, file_name, volume, mesh, directions):
    
    if file_name == 'File_Path_SFM':
        s = 1
    else:
        s = 0.008
    
    num_directions = len(directions)
    # fig = plt.figure(figsize=(15, 5 * num_meshes))
    fig = plt.figure(figsize=(16, 10))
    
    for i, direction  in enumerate(directions):
        # ax = fig.add_subplot(2, 2, i + 1, projection='3d')
        ax = fig.add_subplot(2, 3, i + 1, projection='3d')
        vertices = np.asarray(mesh.vertices)
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=s)
        
        # Set axis labels
        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Z axis')
        
        # Set title
        # ax.set_title(f'3D Mesh Plot {i + 1}')
    
        # Interpret direction as [elev, azim]
        if direction is not None:
            elev, azim = direction
            ax.view_init(elev=elev, azim=azim)
            ax.set_title(f'View: elev={elev}, azim={azim}')
        else:
            ax.set_title('Original View')

    # plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {feet_length}cm - Points: {len(vertices)}\nVolume: {round(volume, 1)}cm3')
    plt.suptitle(f'Subject {subject} - {data_image}\nFeet Length: {feet_length}cm - Points: {len(vertices)}')
    plt.tight_layout()
    # fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.3)

    folder_save_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Details/Feet_Orientation/{file_name}'
    if not os.path.exists(folder_save_path):
        os.makedirs(folder_save_path)

    save_path = f'{folder_save_path}/subject_{subject}_{data_image}.png'
    fig.savefig(save_path, dpi=300)  # Save at high resolution
    plt.close()

    # plt.show()

# Main

In [ ]:
df = pd.read_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_BEST_subjects.csv')
df_subject = pd.read_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Details/data_summary_grouped.csv')

df

In [ ]:
# # Testing
# data_type = 'SFM'
# index = 10

# df_temp = df[df['Type'] == data_type].reset_index(drop=True)
# subject = df_temp['Subject'][index]
# data_image = df_temp['Image Data'][index]
# ply_path = df_temp['File_Path'][index]

# file_name = ply_path.split('/')[-1]

# if 'L' in data_image:
#     column = 'Left foot length (cm)'
# else:
#     column = 'Right foot length (cm)'

# list_feet_length = df_subject[df_subject['Subject ID'] == subject][column]
# list_feet_length = list_feet_length.tolist()[0]
# list_feet_length = list_feet_length.replace(', nan', '').replace('nan', '')
# list_feet_length = ast.literal_eval(list_feet_length)
# feet_length = list_feet_length[0] # First

# print('#'*60)
# print(f'Subject {subject} - {data_image} - Feet length {feet_length}cm')
# print(f'{file_name}')
# print(f'{list_feet_length} - {feet_length}')

# #################################################
# # Load PLY file using Open3D
# #################################################
# if data_type != 'File_Path_SFM':
#     mesh = o3d.io.read_triangle_mesh(ply_path) # Read mesh
# else:
#     mesh = o3d.io.read_point_cloud(ply_path) # Read point dense cloud
#     print('Number of Vertices: ', len(mesh.points))

In [ ]:
df

In [ ]:
list_data_type = [
    # 'SFM', 
    # 'MVS_mesh', 
    # 'MVS_dense', 
    'MVS_refine'
    ]

#################################################
# Run data type
#################################################
for data_type in list_data_type:
    
    #################################################
    # Run subjects
    #################################################
    df_temp = df[df['Type'] == data_type].reset_index(drop=True)
    
    for index, row in df_temp.iterrows():
        subject = df_temp['Subject'][index]
        data_image = df_temp['Image Data'][index]
        ply_path = df_temp['File_Path'][index]
        file_name = ply_path.split('/')[-1]

        if 'L' in data_image:
            column = 'Left foot length (cm)'
        else:
            column = 'Right foot length (cm)'
        
        try:
            list_feet_length = df_subject[df_subject['Subject ID'] == subject][column]
            list_feet_length = list_feet_length.tolist()[0]
            list_feet_length = list_feet_length.replace(', nan', '').replace('nan', '')
            list_feet_length = ast.literal_eval(list_feet_length)
            feet_length = list_feet_length[0] # First
            # feet_length = np.mean(feet_length) # Mean
        except:
            feet_length = 25
        
        
        print('')
        print('')
        print('#'*60)
        print(f'Subject {subject} - {data_image} - Feet length {feet_length}cm')
        print(f'{file_name}')
        print(f'{list_feet_length} - {feet_length}')
        
        try:
            #################################################
            # Load PLY file using Open3D
            #################################################
            if data_type != 'File_Path_SFM':
                mesh = o3d.io.read_triangle_mesh(ply_path) # Read mesh
                print('Number of Vertices: ', len(mesh.vertices))
            else:
                mesh = o3d.io.read_point_cloud(ply_path) # Read point dense cloud
                print('Number of Points: ', len(mesh.points))

                # If more than 2000 points, randomly sample
                points = np.asarray(mesh.points)
                if len(points) > 3000:
                    indices = np.random.choice(len(points), size=2000, replace=False)
                    mesh = mesh.select_by_index(indices)

                # mesh = mesh.voxel_down_sample(voxel_size=0.01)
                print('Number of Points (Down Sample): ', len(mesh.points))

                mesh.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)) # Estimate normals
                mesh.orient_normals_consistent_tangent_plane(100) # Optionally, orient the normals consistently
                
                distances = mesh.compute_nearest_neighbor_distance()
                avg_dist = np.mean(distances)
                radii = [avg_dist, avg_dist * 2]

                radii = [0.05, 0.1, 0.2] # Define radii for the Ball Pivoting algorithm
                mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                    mesh, 
                    o3d.utility.DoubleVector(radii)
                    ) # Apply the Ball Pivoting algorithm

            #################################################
            # Pipeline to edit mesh
            #################################################
            mesh = remove_isolated_vertices(copy.deepcopy(mesh))
            reoriented_mesh = find_floor_plane_and_reorient(copy.deepcopy(mesh))
            cropped_mesh = crop_mesh_z_axis(copy.deepcopy(reoriented_mesh), percentile=21)
            largest_object_mesh = remove_small_objects(copy.deepcopy(cropped_mesh))
            scaled_mesh, point1, point2 = scale_longest_xy_axis(copy.deepcopy(largest_object_mesh), feet_length)
            # rotate_feet_mesh = reorient_feet(copy.deepcopy(scaled_mesh), point1, point2)
            
            try:
                cropped_ankle_mesh = crop_mesh_z_axis_ankle(copy.deepcopy(scaled_mesh), z_threshold=10)
            except:
                cropped_ankle_mesh = copy.deepcopy(scaled_mesh)
                
            # solid_mesh = solidify_mesh_with_fans(copy.deepcopy(cropped_ankle_mesh))

            #################################################
            # Saving results
            #################################################
            path_save = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_edit/{subject}/{data_image}'
            if not os.path.exists(path_save):
                os.makedirs(path_save)
            o3d.io.write_triangle_mesh(f'{path_save}/postprocessed_{file_name}', mesh)

            # Visualize the PLY file
            # o3d.visualization.draw_geometries([reoriented_mesh])
            # plot_mesh(reoriented_mesh)

            #################################################
            # Saving results
            #################################################
            print(f'postprocessed_{file_name}')
            print(f'{path_save}')
            try:
                volume = calculate_volume_file(
                    file_path = f'{path_save}', 
                    file_name = f'postprocessed_{file_name}'
                    )
                error = ''
            except Exception as error:
                print(error)
                volume = 0
            
            print(f'{file_name} - Volume: {round(volume, 3)} unit3')

            #################################################
            # Plot processed meshes
            #################################################
            # plot_all_meshes(
            #     subject, data_image, feet_length, file_name.replace('.ply', ''), volume,
            #     point1, point2,
            #     [mesh, reoriented_mesh, cropped_mesh, 
            #     largest_object_mesh, scaled_mesh, rotate_feet_mesh, 
            #     cropped_ankle_mesh, solid_mesh],
            #     ['Original', 'Reorientate to Align Floor', 'Crop away floor',
            #     'Remove Small Objects', 'Rescaled', 'Rotate Feet (x-y plane)', 
            #     'Crop away ankle', 'Solid Mesh']
            # )
            
            # plot_all_meshes(
            #     subject, data_image, feet_length, file_name.replace('.ply', ''), volume,
            #     0, 0,
            #     [mesh, reoriented_mesh],
            #     ['Original', 'Reorientate to Align Floor']
            # )
            
            plot_all_meshes(
                subject, data_image, feet_length, file_name.replace('.ply', ''), volume,
                point1, point2,
                [mesh, reoriented_mesh, cropped_mesh, 
                largest_object_mesh, scaled_mesh, 
                cropped_ankle_mesh],
                ['Original', 'Reorientate to Align Floor', 'Crop away floor',
                'Remove Small Objects', 'Rescaled',
                'Crop away ankle']
            )
            
            #################################################
            # Plot different orientation of final meshes
            #################################################
            # directions = [
            #     np.array([0, 0]), # front
            #     np.array([0, 90]), # side
            #     np.array([90, 0]), # top
            #     np.array([45, 45]), # diagonal
            # ]

            # directions = [
            #     np.array([0, 0]), np.array([0, 90]), np.array([0, 180]), np.array([0, 270]),
            #     np.array([90, 0]), np.array([90, 90]), np.array([90, 180]), np.array([90, 270]),
            #     np.array([45, 45]), np.array([135, 135]), np.array([225, 225]), np.array([315, 315])
            # ]

            # Define 9 diverse viewing directions
            directions = [
                None, np.array([0, 0]), np.array([0, 90]), np.array([0, 180]),
                np.array([90, 0]), np.array([45, 45])
            ]

            plot_mesh_angles(
                subject, data_image, feet_length, file_name.replace('.ply', ''), volume,
                cropped_ankle_mesh,
                # solid_mesh,
                directions
            )
            
            plt.show()

        except Exception as error:
            print('ERROR')
            print(error)

In [ ]:
cropped_ankle_mesh

In [ ]:
import open3d as o3d
import numpy as np
from collections import defaultdict

def close_mesh_holes(mesh):
    """
    Closes holes in an Open3D mesh by identifying boundary edges and filling them with fan triangulation.
    """
    mesh.compute_adjacency_list()
    triangles = np.asarray(mesh.triangles)
    edges = defaultdict(int)

    # Count edges
    for tri in triangles:
        for i in range(3):
            edge = tuple(sorted((tri[i], tri[(i + 1) % 3])))
            edges[edge] += 1

    # Find boundary edges (edges used only once)
    boundary_edges = [edge for edge, count in edges.items() if count == 1]

    # Group boundary edges into loops
    edge_map = defaultdict(list)
    for u, v in boundary_edges:
        edge_map[u].append(v)
        edge_map[v].append(u)

    loops = []
    visited = set()
    for start in edge_map:
        if start in visited:
            continue
        loop = [start]
        visited.add(start)
        current = start
        while True:
            neighbors = [n for n in edge_map[current] if n not in visited]
            if not neighbors:
                break
            next_v = neighbors[0]
            loop.append(next_v)
            visited.add(next_v)
            current = next_v
        if len(loop) > 2:
            loops.append(loop)

    # Fill each loop with fan triangulation
    vertices = np.asarray(mesh.vertices).tolist()
    triangles = triangles.tolist()
    for loop in loops:
        loop_pts = np.array([vertices[i] for i in loop])
        center = np.mean(loop_pts, axis=0)
        center_idx = len(vertices)
        vertices.append(center.tolist())
        for i in range(len(loop)):
            v1 = loop[i]
            v2 = loop[(i + 1) % len(loop)]
            triangles.append([v1, v2, center_idx])

    closed_mesh = o3d.geometry.TriangleMesh()
    closed_mesh.vertices = o3d.utility.Vector3dVector(np.array(vertices))
    closed_mesh.triangles = o3d.utility.Vector3iVector(np.array(triangles))
    closed_mesh.compute_vertex_normals()
    return closed_mesh


In [ ]:
next_mesh = close_mesh_holes(copy.deepcopy(cropped_ankle_mesh))

In [ ]:
print(f'No of points: {len(np.asarray(next_mesh.vertices))}')

In [ ]:
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

# Load your mesh
mesh = copy.deepcopy(next_mesh)

# Extract vertices and triangles
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create a Poly3DCollection from the mesh triangles
mesh_collection = Poly3DCollection(vertices[triangles], alpha=0.7)
mesh_collection.set_facecolor('lightblue')
mesh_collection.set_edgecolor('k')

# Add the collection to the plot
ax.add_collection3d(mesh_collection)

# Set plot limits
ax.set_xlim(vertices[:, 0].min(), vertices[:, 0].max())
ax.set_ylim(vertices[:, 1].min(), vertices[:, 1].max())
ax.set_zlim(vertices[:, 2].min(), vertices[:, 2].max())

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.tight_layout()
plt.show()
